# Vertical Chat
A sample how to build a chat for small business using:

* GPT 35
* Panel
* OpenAI


This is just a simple sample to start to understand how the OpenAI API works, and how to create Prompts. It Is really far from beign a complete solution.
We are going to introduce some interesting points:

* The roles in a conversation.
* How is the conversations’ memory preserved?

Deeper explanations in the article: [Create Your First Chatbot Using GPT 3.5, OpenAI, Python and Panel.](https://medium.com/towards-artificial-intelligence/create-your-first-chatbot-using-gpt-3-5-openai-python-and-panel-7ec180b9d7f2)

In [1]:
#if you need an API Key from OpenAI
#https://platform.openai.com/account/api-keys

from openai import OpenAI
import os
from dotenv import load_dotenv, find_dotenv
_ = load_dotenv(find_dotenv())

OPENAI_API_KEY  = os.getenv('OPENAI_API_KEY')

In [2]:
client = OpenAI(
    # This is the default and can be omitted
    api_key=OPENAI_API_KEY,
)

def continue_conversation(messages, temperature=0):
    response = client.chat.completions.create(
        model="gpt-3.5-turbo",
        messages=messages,
        temperature=temperature,
    )
    #print(str(response.choices[0].message["content"]))
    return response.choices[0].message.content

In [3]:
def add_prompts_conversation(_):
    #Get the value introduced by the user
    prompt = client_prompt.value_input
    client_prompt.value = ''

    #Append to the context the User prompt.
    context.append({'role':'user', 'content':f"{prompt}"})

    #Get the response.
    response = continue_conversation(context)

    #Add the response to the context.
    context.append({'role':'assistant', 'content':f"{response}"})

    #Update the panels to show the conversation.
    panels.append(
        pn.Row('User:', pn.pane.Markdown(prompt, width=600)))
    panels.append(
        pn.Row('Assistant:', pn.pane.Markdown(response, width=600)))

    return pn.Column(*panels)

In [4]:
#Creating the prompt
#read and understand it.
import panel as pn  # GUI

context = [ {'role':'system', 'content':"""
Act as an OrderBot, you work collecting orders in a delivery only fast food restaurant called
My Dear Frankfurt. \
First welcome the customer, in a very friendly way, then collects the order. \
You wait to collect the entire order, beverages included \
then summarize it and check for a final \
time if everything is ok or the customer wants to add anything else. \
Finally you collect the payment.\
Make sure to clarify all options, extras and sizes to uniquely \
identify the item from the menu.\
You respond in a short, very friendly style. \
The menu includes \
burger  12.95, 10.00, 7.00 \
frankfurt   10.95, 9.25, 6.50 \
sandwich   11.95, 9.75, 6.75 \
fries 4.50, 3.50 \
salad 7.25 \
Toppings: \
extra cheese 2.00, \
mushrooms 1.50 \
martra sausage 3.00 \
canadian bacon 3.50 \
romesco sauce 1.50 \
peppers 1.00 \
Drinks: \
coke 3.00, 2.00, 1.00 \
sprite 3.00, 2.00, 1.00 \
vichy catalan 5.00 \
"""} ]

#Creating the panel.
pn.extension()

panels = []

client_prompt = pn.widgets.TextInput(value="Hi", placeholder='Enter text here…')
button_conversation = pn.widgets.Button(name="talk")

interactive_conversation = pn.bind(add_prompts_conversation, button_conversation)

dashboard = pn.Column(
    client_prompt,
    pn.Row(button_conversation),
    pn.panel(interactive_conversation, loading_indicator=True),
)

dashboard

/var/folders/zf/26z9dj9s4rgb9txx_1j9fkv80000gn/T/ipykernel_95744/3485984612.py:36: UserWarning: Using Panel interactively in VSCode notebooks requires the jupyter_bokeh package to be installed. You can install it with:

   pip install jupyter_bokeh

or:
    conda install jupyter_bokeh

and try again.
  pn.extension()


Column
    [0] TextInput(placeholder='Enter text here…')
    [1] Row
        [0] Button(name='talk')
    [2] ParamFunction(function, _pane=Column, defer_load=False, loading_indicator=True)

# Exercise
 - Complete the prompts similar to what we did in class. 
     - Try at least 3 versions
     - Be creative
 - Write a one page report summarizing your findings.
     - Were there variations that didn't work well? i.e., where GPT either hallucinated or wrong
 - What did you learn?

In [5]:
def moderate_student_forum(comment):
    response = client.moderations.create(input=comment)
    results = response.results[0]
    
    analysis = {
        "is_appropriate": not results.flagged,
        "categories": {
            "bullying": results.categories.harassment,
            "hate_speech": results.categories.hate,
            "violence": results.categories.violence,
            "inappropriate": results.categories.sexual
        },
        "severity_level": "low",
        "action_required": False,
        "recommendation": ""
    }
    
    # Determinar nivel de severidad
    if any([results.categories.harassment, results.categories.hate]):
        analysis["severity_level"] = "high"
        analysis["action_required"] = True
        analysis["recommendation"] = "Remove comment and contact moderator"
    elif results.categories.violence:
        analysis["severity_level"] = "medium"
        analysis["action_required"] = True
        analysis["recommendation"] = "Flag for review"
    else:
        analysis["recommendation"] = "Approved"
    
    return analysis

# Pruebas
student_comments = [
    "This homework assignment was really helpful!",
    "You're so dumb, you'll never pass this class",
    "I disagree with your answer but let's discuss why"
]

for comment in student_comments:
    result = moderate_student_forum(comment)
    print(f"\nComment: {comment}")
    print(f"Appropriate: {result['is_appropriate']}")
    print(f"Severity: {result['severity_level']}")
    print(f"Action Required: {result['action_required']}")
    print(f"Recommendation: {result['recommendation']}")


Comment: This homework assignment was really helpful!
Appropriate: True
Severity: low
Action Required: False
Recommendation: Approved

Comment: You're so dumb, you'll never pass this class
Appropriate: False
Severity: high
Action Required: True
Recommendation: Remove comment and contact moderator

Comment: I disagree with your answer but let's discuss why
Appropriate: True
Severity: low
Action Required: False
Recommendation: Approved


In [6]:
def moderate_tech_blog(content):
    response = client.moderations.create(input=content)
    results = response.results[0]
    
    analysis = {
        "is_appropriate": not results.flagged,
        "categories": {
            "harassment": results.categories.harassment,
            "hate_speech": results.categories.hate,
            "unsafe_content": results.categories.violence
        },
        "content_rating": "safe",
        "needs_review": False,
        "feedback": ""
    }
    
    # Evaluar contenido
    if results.categories.harassment or results.categories.hate:
        analysis["content_rating"] = "unsafe"
        analysis["needs_review"] = True
        analysis["feedback"] = "Contains inappropriate content"
    elif results.categories.violence:
        analysis["content_rating"] = "questionable"
        analysis["needs_review"] = True
        analysis["feedback"] = "Review for technical accuracy"
    
    return analysis

# Pruebas
tech_posts = [
    "Review of the latest security features in Linux",
    "This company's products are terrible and they're all scammers",
    "Comparing different cloud providers' performance"
]

for post in tech_posts:
    result = moderate_tech_blog(post)
    print(f"\nPost: {post}")
    print(f"Rating: {result['content_rating']}")
    print(f"Needs Review: {result['needs_review']}")
    print(f"Feedback: {result['feedback']}")


Post: Review of the latest security features in Linux
Rating: safe
Needs Review: False
Feedback: 

Post: This company's products are terrible and they're all scammers
Rating: safe
Needs Review: False
Feedback: 

Post: Comparing different cloud providers' performance
Rating: safe
Needs Review: False
Feedback: 


In [7]:
def moderate_learning_platform(content):
    response = client.moderations.create(input=content)
    results = response.results[0]
    
    analysis = {
        "is_appropriate": not results.flagged,
        "categories": {
            "inappropriate_language": results.categories.harassment,
            "discriminatory": results.categories.hate,
            "unsafe": results.categories.violence
        },
        "educational_value": "high",
        "moderation_needed": False,
        "suggestions": []
    }
    
    # Evaluar contenido educativo
    if any([results.categories.harassment, results.categories.hate, results.categories.violence]):
        analysis["educational_value"] = "low"
        analysis["moderation_needed"] = True
        analysis["suggestions"].append("Review content for educational appropriateness")
    
    return analysis

# Pruebas
learning_content = [
    "Can someone help me understand this math problem?",
    "This topic is stupid and so is everyone studying it",
    "Here's a detailed explanation of the concept"
]

for content in learning_content:
    result = moderate_learning_platform(content)
    print(f"\nContent: {content}")
    print(f"Educational Value: {result['educational_value']}")
    print(f"Needs Moderation: {result['moderation_needed']}")
    if result['suggestions']:
        print(f"Suggestions: {result['suggestions']}")


Content: Can someone help me understand this math problem?
Educational Value: high
Needs Moderation: False

Content: This topic is stupid and so is everyone studying it
Educational Value: high
Needs Moderation: False

Content: Here's a detailed explanation of the concept
Educational Value: high
Needs Moderation: False


In [8]:
def test_all_moderation_systems():
    print("=== Testing Student Forum Moderation ===")
    for comment in student_comments:
        result = moderate_student_forum(comment)
        print(f"\nComment: {comment}")
        print(f"Result: {result['recommendation']}")
    
    print("\n=== Testing Tech Blog Moderation ===")
    for post in tech_posts:
        result = moderate_tech_blog(post)
        print(f"\nPost: {post}")
        print(f"Result: {result['feedback']}")
    
    print("\n=== Testing Learning Platform Moderation ===")
    for content in learning_content:
        result = moderate_learning_platform(content)
        print(f"\nContent: {content}")
        print(f"Result: {'Needs moderation' if result['moderation_needed'] else 'Approved'}")

# Ejecutar todas las pruebas
test_all_moderation_systems()

=== Testing Student Forum Moderation ===

Comment: This homework assignment was really helpful!
Result: Approved

Comment: You're so dumb, you'll never pass this class
Result: Remove comment and contact moderator

Comment: I disagree with your answer but let's discuss why
Result: Approved

=== Testing Tech Blog Moderation ===

Post: Review of the latest security features in Linux
Result: 

Post: This company's products are terrible and they're all scammers
Result: 

Post: Comparing different cloud providers' performance
Result: 

=== Testing Learning Platform Moderation ===

Content: Can someone help me understand this math problem?
Result: Approved

Content: This topic is stupid and so is everyone studying it
Result: Approved

Content: Here's a detailed explanation of the concept
Result: Approved
